In [ ]:
import matplotlib.pyplot as plt
from obspy import Stream,read
import UNAVCO_RT
import numpy as np
import os

stas_file = np.genfromtxt('/hdd/Ridgecrest/summer23/GNSS_stas.txt', dtype = str) 
stas = stas_file[:,2].tolist() # List of station names

dates_array = np.genfromtxt('/hdd/Ridgecrest/summer23/GNSS_dates.txt', dtype = str) 
dates = dates_array.tolist() # List of dates for which we have UNAVCO RT data: format YYYYMMDD

chans = ['e', 'n', 'u']

project = 'summer23'
path_to_files = '/hdd/Ridgecrest/rt_solutions/' # Where are the UNAVCO daily txt files?
project_dir = '/hdd/Ridgecrest/' + project + '/'
write_mseed_path = project_dir + 'daily_mseeds/' # Where to save the daily mseeds we'll make by parsing?
figure_path = project_dir + 'wvf_figures/' # If you want to check the data, save the figures here

if os.path.isdir(write_mseed_path):
    pass
else:
    os.makedirs(write_mseed_path)
    os.makedirs(figure_path)
    
small_test = False

if small_test:
    
    dates = ['20190606', '20190731']
    stas = ['BEPK', 'CCCC']

In [ ]:
for i in range(0, len(dates)):
    
    date = dates[i]
    
    for k in range(0, len(stas)):
        
        sta = stas[k]
        
        write_mseed_sta_path = write_mseed_path + sta + '/'
        figure_sta_path = figure_path + sta + '/'
        
        if os.path.isdir(write_mseed_sta_path):
            pass
        else:
            os.makedirs(write_mseed_sta_path)
            os.makedirs(figure_sta_path)
        
        try:
        
            df = UNAVCO_RT.parse(path_to_files + sta + '/' + sta + '_rtx_' + date + '.txt')

            n, e, u = UNAVCO_RT.unavco2neu(df)

            stn, ste, stu = UNAVCO_RT.pandas2obspy(df,n,e,u)

            # stn_merge = stn.merge(fill_value=999)
            # ste_merge = ste.merge(fill_value=999)
            # stu_merge = stu.merge(fill_value=999)

            # stn_merge = stn.merge(fill_value='interpolate')
            # ste_merge = ste.merge(fill_value='interpolate')
            # stu_merge = stu.merge(fill_value='interpolate')

            stn.write(write_mseed_sta_path + sta + '.n.' + date + '.mseed', format = 'MSEED')
            ste.write(write_mseed_sta_path + sta + '.e.' + date + '.mseed', format = 'MSEED')
            stu.write(write_mseed_sta_path + sta + '.u.' + date + '.mseed', format = 'MSEED')

            print('Station ' + sta + ', date ' + date + ': SUCCESS')

            for chan in chans:

                st = Stream()
                st = read(write_mseed_sta_path + sta + '.' + chan + '.' + date + '.mseed')

                st_merge = st.merge(fill_value='interpolate')

                times = st_merge[0].times()
                data = st_merge[0].data

                plt.figure()
                plt.plot(times,data,label=chan)
                plt.title('UNAVCO Real-Time Data, station ' + sta + ', channel ' + chan + ', date ' + date)
                plt.xlabel('Time (s)')
                plt.ylabel('Displacement (m)')

                plt.savefig(figure_sta_path + sta + '.' + chan + '.' + date + '.png', format = 'PNG')

        except:
           
            print('Station ' + sta + ', date ' + date + ': ERROR')
    
    